In [2]:
import scipy.io 
import numpy as np
import matplotlib.pyplot as plt
def anorm(x):
    '''Calculate L2 norm over the last array dimention'''
    return np.sqrt((x * x).sum(-1))


def calc_energy_tgv(u, v):
    energy_tgv = anorm(epsilon(nabla(u))).sum()
    energy_tgv_dual = anorm(epsilon(v)).sum()
    return energy_tgv, energy_tgv_dual


def nabla(I):
    h, w = I.shape
    G = np.zeros((h, w, 2), I.dtype)
    G[:, :-1, 0] -= I[:, :-1]
    G[:, :-1, 0] += I[:, 1:]
    G[:-1, :, 1] -= I[:-1]
    G[:-1, :, 1] += I[1:]
    return G


def nablaT(G):
    h, w = G.shape[:2]
    I = np.zeros((h, w), G.dtype)
    # note that we just reversed left and right sides
    # of each line to obtain the transposed operator
    I[:, :-1] -= G[:, :-1, 0]
    I[:, 1:] += G[:, :-1, 0]
    I[:-1] -= G[:-1, :, 1]
    I[1:] += G[:-1, :, 1]

    return I


def epsilon(I):
    h, w, _ = I.shape
    G = np.zeros((h, w, 4), I.dtype)
    G[:, :-1, 0] -= I[:, :-1, 0]  # xdx
    G[:, :-1, 0] += I[:, 1:, 0]
    G[:-1, :, 1] -= I[:-1, :, 0]  # xdy
    G[:-1, :, 1] += I[1:, :, 0]
    G[:, :-1, 2] -= I[:, :-1, 1]  # ydx
    G[:, :-1, 2] += I[:, 1:, 1]
    G[:-1, :, 3] -= I[:-1, :, 1]  # ydy
    G[:-1, :, 3] += I[1:, :, 1]
    return G


def epsilonT(G):
    h, w, _ = G.shape
    I = np.zeros((h, w, 2), G.dtype)
    # note that we just reversed left and right sides
    # of each line to obtain the transposed operator
    I[:, :-1, 0] -= G[:, :-1, 0]  # xdx
    I[:, 1:, 0] += G[:, :-1, 0]
    I[:-1, :, 0] -= G[:-1, :, 1]  # xdy
    I[1:, :, 0] += G[:-1, :, 1]
    I[:, :-1, 1] -= G[:, :-1, 2]  # ydx
    I[:, 1:, 1] += G[:, :-1, 2]
    I[:-1, :, 1] -= G[:-1, :, 3]  # ydy
    I[1:, :, 1] += G[:-1, :, 3]

    return I


def project_nd(P, r):
    '''perform a pixel-wise projection onto R-radius balls'''
    if r == 0:
        return np.zeros_like(P)
    nP = np.maximum(1.0, anorm(P) / r)
    return P / nP[..., np.newaxis]
# tgv iteration
NBINS = 8
EMPTY_VALUE = 0

def append_observation(bins_ns, observation, mask):
    for j in range(NBINS):
        cj_from = j*32
        cj_to   = j*32 + 31
        bin_mask = np.logical_and(mask, np.logical_and(observation >= cj_from, observation < cj_to))
        bin_slice = bins_ns[:, :, j]
        bin_slice[bin_mask] += 1


def prox(v, observations, Ws, tau, labmda_data):
    res = np.median(np.dstack(observations+ [v+tau*lambda_data*Ws[k] for k in range(len(observations))]), axis=-1)
    return res
            


def iter_tgv(u, v, p, q, observations, tau, lambda_tv, lambda_tgv, lambda_data, Ws):
    tau_u, tau_v, tau_p, tau_q = tau, tau, tau, tau

    un = prox(u - tau_u * lambda_tv * nablaT(p), observations, Ws, tau_u, lambda_data)
    vn = v + tau_v * (lambda_tgv * (-epsilonT(q)) + lambda_tv * p)

    pn = project_nd(p + tau_p * lambda_tv * (nabla(2 * un - u) - (2 * vn - v)), lambda_tv)

    qn = project_nd(q + tau_q * lambda_tgv * epsilon(2 * vn - v), lambda_tgv)
    return un, vn, pn, qn
def solve(noisy_images, tau, lambda_tv, lambda_tgv, lambda_data, iter_n=101):
    # Initialize empty lists to store denoised images and energies
    denoised_images = []
    energies_tv = []
    energies_tgv_dual = []
    
    # Iterate over each noisy image
    for noisy_image in noisy_images:
        # Initialize variables for TGV denoising
        height, width = noisy_image.shape
        u = np.zeros((height, width), dtype=np.float32)
        u[:, :] = noisy_image
        v = nabla(u)
        p = nabla(u)
        q = epsilon(v)
        energies_tv_iter = []
        energies_tgv_dual_iter = []
        
        # Calculate initial energies
        tgv_energy, tgv_energy_dual = calc_energy_tgv(u, v)
        energies_tv_iter.append(tgv_energy)
        energies_tgv_dual_iter.append(tgv_energy_dual)
        
        # Perform TGV denoising iterations
        for i in range(iter_n):
            u, v, p, q = iter_tgv(u, v, p, q, [noisy_image], tau, lambda_tv, lambda_tgv, lambda_data, [np.ones_like(noisy_image)])
            tgv_energy, tgv_energy_dual = calc_energy_tgv(u, v)
            energies_tv_iter.append(tgv_energy)
            energies_tgv_dual_iter.append(tgv_energy_dual)
        
        # Store the denoised image and energies for this noisy image
        denoised_images.append(u)
        energies_tv.append(energies_tv_iter)
        energies_tgv_dual.append(energies_tgv_dual_iter)
    
    # Return the denoised images and energies
    return denoised_images, energies_tv, energies_tgv_dual


In [3]:
# Define the specific SNRs to display
snrs = [10, 5, 3, 1, 0]
tau         = 1/(8**0.5) / 4/8
lambda_tv   = 1.0
lambda_tgv  = 1.0
lambda_data = 1.0
lambda_tv   /= lambda_data
lambda_tgv  /= lambda_data
iter_n = 100  # Number of iterations for the denoising algorithm

# Iterate over each SNR
for snr in snrs:
    # Load the MAT file
    mat = scipy.io.loadmat(f'/Users/dolorious/Downloads/simulation-1/GroundTruth_gaussian_SNR{snr}.mat')

    # Extract noisy image data for the specified SNR
    noisy_image_data = mat['data_gaussian']

    # Assuming 'data_gaussian' is a 3D array, with the third dimension being different noise conditions
    conditions = noisy_image_data.shape[2]

    plt.figure(figsize=(20, 8))  # Adjust the figure size as needed

    for j in range(conditions):
        # Extract the noisy image for the jth condition
        noisy_image = noisy_image_data[:, :, j].squeeze()

        # Convert to grayscale if necessary
        if noisy_image.ndim == 3 and noisy_image.shape[2] == 3:
            noisy_image = noisy_image.mean(axis=2)

        # Ensure noisy_image is a numpy array
        noisy_image = np.array(noisy_image)

        # Print debug information about noisy_image
        print(f"Noisy image shape: {noisy_image.shape}")

        # Determine the shape of the noisy image
        if len(noisy_image.shape) != 2:
            # Handle cases where noisy_image is not a 2D array
            print("Noisy image is not a 2D array")
            continue  # Skip processing this image

        # Now we have a defined image, call the function
        denoised_image, _, _, _, _ = solve(noisy_image,  lambda_tv, lambda_tgv, lambda_data, iter_n)

        # Plot the original noisy image
        plt.subplot(2, conditions, j + 1)  # 2 rows, N columns, jth subplot
        plt.imshow(noisy_image, cmap='gray')  # Display noisy image in grayscale
        plt.title('Noisy Image')
        plt.axis('off')

        # Plot the denoised image
        plt.subplot(2, conditions, j + conditions + 1)  # 2 rows, N columns, (j+N)th subplot
        plt.imshow(denoised_image, cmap='gray')  # Display denoised image in grayscale
        plt.title('Denoised Image')
        plt.axis('off')
    
    plt.tight_layout(pad=3.0, h_pad=3.0, w_pad=1.0)  # Adjust horizontal and vertical padding between subplots

    # Adjust the top margin to make space for the super title
    plt.subplots_adjust(top=0.85)

    # Add a super title and adjust its position
    plt.suptitle(f'Noisy Images - SNR {snr} with TV Denoising', fontsize=16, y=0.98)

    # Show plot
    plt.show()



Noisy image shape: (90, 90)


ValueError: not enough values to unpack (expected 2, got 1)

<Figure size 2000x800 with 0 Axes>